In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import re

2023-02-03 09:03:18.690864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 09:03:18.692388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 09:03:18.693434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 09:03:18.707922: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
df = pd.read_csv("/kaggle/input/c-code-from-project-codenet/function_swap_samples_revised.txt",sep='\t')#/kaggle/input/my-data
#df.columns = ["function_name","arg1","arg2","arg_type","param1","param2","labels"]

In [5]:
df.to_csv('hello.csv')

In [6]:
df.isna().sum()

function_name      646
arg1               702
arg2               702
arg_type             0
param1           64808
param2           64778
labels               0
dtype: int64

In [9]:
# df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],axis=1,inplace=True)
# df

In [10]:
# df.isna().sum()

In [7]:
df["function_name"] = df["function_name"].fillna(' ')
df["arg1"]=df["arg1"].fillna(' ')
df["arg2"]=df["arg2"].fillna(' ')
df["arg_type"]=df["arg_type"].fillna(' ')
df["param1"]=df["param1"].fillna(' ')
df["param2"]=df["param2"].fillna(' ')

In [8]:
df.isna().sum()

function_name    0
arg1             0
arg2             0
arg_type         0
param1           0
param2           0
labels           0
dtype: int64

In [9]:
# df.dropna(inplace=True)

In [10]:
df.isna().sum()

function_name    0
arg1             0
arg2             0
arg_type         0
param1           0
param2           0
labels           0
dtype: int64

In [11]:
df['total'] = df["function_name"] + ' '+ df["arg1"] + ' '+df["arg2"] + ' '+ df["arg_type"] + ' '+df["param1"]+ ' '+ df["param2"]

In [12]:
df.drop(["function_name","arg1","arg2","arg_type","param1","param2"],axis=1,inplace=True)

In [13]:
df = df.sample(frac = 1) # shuffling the dataset
df.reset_index(drop=True,inplace=True)

In [14]:
df.shape

(87690, 2)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(49788, 2)

In [17]:
df

,labels,total
0,1,WREG32 0x500 index_reg int
1,0,calloc ncolumns sizeof void * __nmemb __size
2,1,PCU_ASSERT_EQUAL ret2 ret1 int
3,1,__builtin_expect 1 __extension__ long
4,1,gpio_set_value 1 59 int
...,...,...
87680,1,time_after mad_send_wr mad_agent_priv int
87683,0,strcmp cmd --help int
87685,1,max size 1U int
87687,1,ERROR strerror fdopen: %s int


In [20]:
df.to_csv('my_csv.csv')

In [19]:
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, T5ForConditionalGeneration

In [26]:
# tokenizer = AutoTokenizer.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')
# classifier = AutoModelForSequenceClassification.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')

In [27]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
classifier = AutoModel.from_pretrained("microsoft/codebert-base")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [29]:
m = len(df)
sample = int(m/10)

In [30]:
train_df = df[sample:]
test_df = df[:sample]

In [31]:
# InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
#                                                           text_a = x[code_snippet], 
#                                                           text_b = None,
#                                                           label = x[label_of_code_snippet])

In [32]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [33]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, 
                                                                           test_df, 
                                                                           'total', 
                                                                           'labels')

In [34]:
train_df

,labels,total
5772,1,writel_relaxed VCAP_OFFSET 0x00010fff int
5773,0,fopen filename wb FILE * __filename __modes
5775,0,nodes_identical_1 p q int
5776,0,crypto_sign_keypair pk sk int
5777,1,dup2 1 data int __fd __fd2
...,...,...
87676,1,logit ra %s int
87678,1,device_create_file sdev & int
87679,0,subseg_set 3 0 int
87682,1,v4l2_fh_init dev & int


In [35]:
test_df

,labels,total
0,1,strcmp interrupt name int __s1 __s2
1,0,g_dbl f_dbl cls_struct_align
2,1,zlog_debug ospf6_route_table_name %s: route ad...
3,0,queue_work whc & int
4,1,purple_debug_info Login replies OK; everything...
...,...,...
5766,0,strcpy event jobRecord char *
5768,0,strcmp d .. int
5769,0,assertEqualInt 4 archive_read_data int
5770,0,x '\002' 1 int


In [36]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [37]:
code_snippet = 'total'
label_of_code_snippet = 'labels'

In [38]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [39]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

AttributeError: 'RobertaModel' object has no attribute 'compile'

In [ ]:
history = classifier.fit(train_data, epochs=10, validation_data=validation_data,verbose=1,batch_size=256,steps_per_epoch=175)

In [ ]:
import keras
from matplotlib import pyplot as plt

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [35]:
my_predict = classifier.predict(test_df)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
param_grid = {
    "per_gpu_batch_size": [16, 32,64],
    "learning_rate": [5e-6, 1e-5, 5e-5],
    "num_epochs": [5, 10, 25,40]
    "steps_per_epoch":[2128,1064,425,266]
}

In [ ]:
from sklearn import model_selection

In [ ]:
model = model_selection.GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=1,
    n_jobs=2,
    cv=5
)

In [ ]:
model.fit(train_data)